In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os, shutil
!pip install -q efficientnet
import efficientnet.keras as efn
import tensorflow_probability as tfp


In [ ]:
tfd = tfp.distributions
AUTOTUNE = tf.data.experimental.AUTOTUNE
IMG_SIZE = 299
BATCH_SIZE = 16
EPOCHS = 7
FOLDS = 6
FILES_LIST = tf.io.gfile.glob('../input/plant-pathology-512x512-augmented/*.tfrecords')
FILES_LIST.sort()
FOLDS_LIST=dict()
STEPS_PER_EPOCH = dict()
for i in range(FOLDS):
    FOLDS_LIST['fold_'+str(i)] = dict({'train':FILES_LIST[:i]+FILES_LIST[i+1:], 'val':FILES_LIST[i]})

total_rows = 0
for k in FILES_LIST:
    df = pd.read_csv(k[:-9]+'csv')
    total_rows += df.shape[0]

STEPS_PER_EPOCH = int((total_rows*0.7)/BATCH_SIZE)    
TFRECS_FORMAT={'image': tf.io.FixedLenFeature([], tf.string),
                      'image_name': tf.io.FixedLenFeature([], tf.string),
                      'target': tf.io.FixedLenFeature([6,], tf.float32)}

In [ ]:
def mixup(entry1,entry2):
    image1,label1 = entry1
    image2,label2 = entry2
    
    alpha = [0.2]
    dist = tfd.Beta(alpha, alpha)
    l = dist.sample(1)[0][0]
    
    img = l*image1+(1-l)*image2
    lab = l*label1+(1-l)*label2
    
    return img, lab

In [ ]:
def decode_entry(entry):
    return tf.cast(tf.reshape(tf.io.decode_raw(entry['image'],tf.uint8),[512,512,3]),tf.float32)/255.,entry['target']

@tf.function
def resize(image):
    return tf.image.resize(image,[IMG_SIZE,IMG_SIZE])

def make_datasets(fold_num):
    train_TFRecDataset=tf.data.TFRecordDataset(FOLDS_LIST['fold_'+str(fold_num)]['train'])
    train_Dataset = train_TFRecDataset.map(lambda example:tf.io.parse_example(example,TFRECS_FORMAT))
    train_Dataset = train_Dataset.map(lambda entry: decode_entry(entry))
    train_Dataset = train_Dataset.map(lambda image,target: (resize(image),target))
    train_Dataset1 = train_Dataset.shuffle(128)
    train_Dataset2 = train_Dataset.shuffle(128)
    train_Dataset = tf.data.Dataset.zip((train_Dataset1, train_Dataset2))
    train_Dataset = train_Dataset.map(lambda entry1,entry2: mixup(entry1,entry2))
    
    val_TFRecDataset=tf.data.TFRecordDataset([FOLDS_LIST['fold_'+str(fold_num)]['val']])
    val_Dataset = val_TFRecDataset.map(lambda example:tf.io.parse_example(example,TFRECS_FORMAT))
    val_Dataset = val_Dataset.map(lambda entry: decode_entry(entry))
    val_Dataset = val_Dataset.map(lambda image,target: (resize(image),target) )
    
    return train_Dataset.batch(BATCH_SIZE).repeat().prefetch(AUTOTUNE),val_Dataset.batch(BATCH_SIZE).repeat().prefetch(AUTOTUNE) ####

In [ ]:
def make_model():
    model = tf.keras.Sequential()
    efn_model = efn.EfficientNetB5(weights='noisy-student',input_shape=(IMG_SIZE,IMG_SIZE,3))
    base_model = tf.keras.models.Model(inputs=efn_model.input, outputs=efn_model.layers[-3].output)
    base_model.trainable = True
#     base_model = tf.keras.applications.Xception(include_top=False,
#                                             weights="imagenet",
#                                             input_tensor=None,
#                                             input_shape=(299,299,3),
#                                             pooling='max')
    base_model.trainable = True
    model.add(tf.keras.Input((299,299,3)))
    model.add(base_model)
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dense(10,activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dense(6,activation='softmax'))

    model.compile(optimizer='nadam',
                 loss=tf.keras.losses.CategoricalCrossentropy(),
                 metrics=[tf.keras.metrics.CategoricalAccuracy()])
    return model

In [ ]:
for fold_num in range(FOLDS):
    
    print('#'*10)
    print('Training fold ',fold_num)
    print('#'*10)
    
    train_ds,val_ds = make_datasets(fold_num)
    
    filepath = "fold_%i_effnet5_mixup_aug.hdf5"%fold_num
    save_checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss",
                                                    factor=0.1,
                                                    patience=2,
                                                    verbose=1,)

    
    model = make_model()
    
    model.fit(train_ds,
             validation_data = val_ds,
             epochs = EPOCHS,
             steps_per_epoch = STEPS_PER_EPOCH,
             validation_steps = STEPS_PER_EPOCH,
             callbacks = [save_checkpoint],
             verbose = 1)